In [35]:
import os
import pandas as pd
from lxml import etree
from os import walk


RAW_DIR = "Grayson_CAD_RAWDIR"

BASE_URL = "https://grayson.propertytaxpayments.net"

selectors = {
    "full_name": "//div/div/span[@class='font-weight-bold']",
    "mailing_address": "//div/div/span[@class='font-weight-bold']/../following-sibling::*",
    "type": "//small[contains(text(), 'Type')]/../following-sibling::div/span",
    "ownership": "//small[contains(text(), 'Ownership %')]/../following-sibling::*",
    "property_address": "//small[contains(text(), 'Location')]/../following-sibling::*",
    "legal_description": "//small[contains(text(), 'Legal')]/../following-sibling::div/p",
    "total_due": "//div/div/h5[@class='font-weight-light']",
}

In [36]:
# Output File
output_file = "GCAD_COMPLETED.xlsx"
master_data = []

case_url = (
    "https://grayson.propertytaxpayments.net/Accounts/AccountDetails?taxAccountNumber="
)

# We are reading the files from the local folder
for dirpath, dirnames, filenames in walk(RAW_DIR):
    # We are looking at the filename in 'filename'
    for filename in filenames:
        if filename.endswith(".html"):
            # open the file and read the data
            file_path = os.path.join(dirpath, filename)
            print(f"Processing file: {file_path}")
            with open(file_path, "r") as f:
                raw_html = f.read()

            parser = etree.HTMLParser()
            tree = etree.fromstring(raw_html, parser)

            record = {}
            # Use the selector variable to find the xpath element and fetch text
            for name, selector in selectors.items():
                elements = tree.xpath(selector)
                if elements:
                    # If the element is an lxml Element, get its text content
                    # text = [elem.text.strip() if elem.text else '' for elem in elements ]
                    text = [
                        (
                            elem.xpath("text()")[0].strip()
                            if elem.xpath("text()")[0]
                            else ""
                        )
                        for elem in elements
                    ]
                    record[name] = " ".join(text)
                else:
                    print(f"{name}: Not found")

            # add further payment history four columns
            payment_history = []
            selector = "(//table[@class='table table-striped small '])[2]//tr"
            elements = tree.xpath(selector)
            if elements:
                # If the element is an lxml Element, get its text content
                for elem in elements:
                    if elem.xpath("td"):
                        tds = elem.xpath("td")
                        receipt_record = [
                            " ".join(td.xpath("text()")) for td in tds[:-1]
                        ]
                        if receipt_record[-1] == "":
                            continue
                        receipt_link = (
                            BASE_URL + tds[-1].xpath("a/@href")[0]
                            if tds[-1].xpath("a/@href")
                            else ""
                        )
                        receipt_record.append(receipt_link)
                        payment_history.append(receipt_record)
            latest_receipt = payment_history.pop(0) if payment_history else []
            last_payment_amount = latest_receipt[1]
            if last_payment_amount != "$0.00":
                record["last_payment_date"] = latest_receipt[0]
                record["last_payment_amount"] = last_payment_amount
                record["last_tax_year_paid"] = latest_receipt[2]
                record["last_payer"] = latest_receipt[3]
                record["last_receipt_link"] = (
                    latest_receipt[4] if len(latest_receipt) > 4 else ""
                )
                record["payment_history"] = "; ".join(
                    [str(item) for item in payment_history]
                )
            else:
                record["last_payment_date"] = ""
                record["last_payment_amount"] = ""
                record["last_tax_year_paid"] = ""
                record["last_payer"] = ""
                record["last_receipt_link"] = ""
                record["payment_history"] = []

            record["case_url"] = case_url + filename.split(".")[0].lstrip("R")
            master_data.append(
                {"property_id": filename.split(".")[0].lstrip("R"), **record}
            )

Processing file: Grayson_CAD_RAWDIR/R100014.html
Processing file: Grayson_CAD_RAWDIR/R100055.html
Processing file: Grayson_CAD_RAWDIR/R100110.html
Processing file: Grayson_CAD_RAWDIR/R100096.html
Processing file: Grayson_CAD_RAWDIR/R100079.html
Processing file: Grayson_CAD_RAWDIR/R100080.html
Processing file: Grayson_CAD_RAWDIR/R100018.html
Processing file: Grayson_CAD_RAWDIR/R100075.html
Processing file: Grayson_CAD_RAWDIR/R100022.html
Processing file: Grayson_CAD_RAWDIR/R100126.html
Processing file: Grayson_CAD_RAWDIR/R100063.html
Processing file: Grayson_CAD_RAWDIR/R100127.html
Processing file: Grayson_CAD_RAWDIR/R100062.html
Processing file: Grayson_CAD_RAWDIR/R100131.html
Processing file: Grayson_CAD_RAWDIR/R100074.html
Processing file: Grayson_CAD_RAWDIR/R100058.html
Processing file: Grayson_CAD_RAWDIR/R100019.html
Processing file: Grayson_CAD_RAWDIR/R100081.html
Processing file: Grayson_CAD_RAWDIR/R100097.html
Processing file: Grayson_CAD_RAWDIR/R100054.html
Processing file: Gra

In [37]:
master_df = pd.DataFrame(master_data)

In [38]:
master_df.head()

,property_id,full_name,mailing_address,type,ownership,property_address,legal_description,total_due,last_payment_date,last_payment_amount,last_tax_year_paid,last_payer,last_receipt_link,payment_history,case_url
0,100014,LAWAYNE E JONES LLC,"PO BOX 1185 DUNCAN, OK 73534",Real,100.000000,DELAWARE BEND RD,"G-1069 STEWART SAMUEL A-G1069, ACRES 291.5",$0.00,12/12/2024,"$3,172.86",2024,LAWAYNE E JONES LLC,https://grayson.propertytaxpayments.net/Receip...,"['12/29/2023', '$3,196.46', '2023', 'LAWAYNE E...",https://grayson.propertytaxpayments.net/Accoun...
1,100055,GARZA MICHAEL ERIC ETUX KANDI,"5960 W PARKER RD STE 278 PMB 107 PLANO, TX 75093",Real,100.000000,144 DANIEL LN POTTSBORO,"G-0016 ALBERTY DAVID A-G0016, ACRES 0.979",$0.00,12/17/2024,"$15,652.70",2024,CORELOGIC REAL ESTATE TAX SERVICE,https://grayson.propertytaxpayments.net/Receip...,"['1/9/2024', '$15,606.11', '2023', 'CORELOGIC ...",https://grayson.propertytaxpayments.net/Accoun...
2,100110,BARFIELD BOURDON ETUX MELANIE,"3651 W ELDORADO PKWY MCKINNEY, TX 75070-4227",Real,100.000000,44 PARK LN POTTSBORO,"GLEN EDEN PARK S/D, LOT 21, ACRES .1808",$0.00,12/30/2024,"$9,023.88",2024,BOURDON ETUX MELANIE BARFIELD,https://grayson.propertytaxpayments.net/Receip...,"['1/2/2024', '$9,328.02', '2023', 'BOURDON BAR...",https://grayson.propertytaxpayments.net/Accoun...
3,100096,MLINARICH BEJA ETVIR ROBERT P,"119 BEACHVIEW CIR POTTSBORO, TX 75076-3829",Real,100.000000,119 BEACHVIEW CIR POTTSBORO,"GLEN EDEN PARK S/D, LOT 8, ACRES .1722",$0.00,12/4/2024,"$1,046.34",2024,MLINARICH BEJA ETVIR ROBERT P,https://grayson.propertytaxpayments.net/Receip...,"['12/29/2023', '$165.98', '2023', 'MLINARICH B...",https://grayson.propertytaxpayments.net/Accoun...
4,100079,RINKER SUSAN K,"360 YOSEMITE DR PROSPER, TX 75078-9071",Real,100.000000,44 HUGHES DR POTTSBORO,"GLEN EDEN NORTH LAKE, LOT 34 & 35, ACRES .2296",$0.00,12/6/2024,"$8,737.18",2024,RINKER SUSAN K,https://grayson.propertytaxpayments.net/Receip...,"['12/21/2023', '$8,826.73', '2023', 'RINKER SU...",https://grayson.propertytaxpayments.net/Accoun...


In [ ]:
try:
    master_df.to_excel(output_file, index=False)
    print(f"Data successfully saved to {output_file}")
except PermissionError:
    print(f"Error: Cannot write to {output_file}. The file may be open in another application.")
    print("Please close the file and try again, or use a different filename.")
except Exception as e:
    print(f"An error occurred while saving the file: {e}")